In [1]:
import pandas as pd
import numpy as np

In [9]:
data = pd.read_csv('../data/labeled_post.csv')

In [10]:
data.head()

,title,body,subreddit,url,score,num_comments,upvote_ratio,created_utc,author,is_original_content,edited,link_flair_text,total_awards_received,gilded,search_query
0,Have I hit the jackpot?,Hi REP community! I’m an amateur female creato...,RealEstatePhotography,https://www.reddit.com/gallery/1kmomx4,0,10,0.23,1.747252e+09,Kaenatseth,False,False,NaN,0,0,recommend software for
1,Travel MRI Tech assignment in Huntington WVa w...,Hi everyone.\n\nSiemens is looking for an expe...,MRI,https://www.reddit.com/r/MRI/comments/1kmqv5e/...,2,1,1.00,1.747258e+09,Fuzzy_Barracuda_7160,False,False,NaN,0,0,looking for program to
2,Sportradar goes full platform guru in Brazil. ...,"Oh, look who's trying to reinvent themselves. ...",cataclysms,https://www.reddit.com/r/cataclysms/comments/1...,2,0,1.00,1.747142e+09,cracking_cracker,False,False,NaN,0,0,tired of manually
3,Help Build the Ultimate Get Paid To Play Offer...,Welcome to the Megalist of Games - our growing...,GetPaidToPlay,https://www.reddit.com/r/GetPaidToPlay/comment...,12,11,1.00,1.747235e+09,TightAsF_ck,False,True,Guide :GetPaidToPlay:,0,0,app to track
4,"No cameras working DS-7616NI-SE/P, no POE ligh...",I have hit a wall with the troubleshooting her...,Hikvision,https://www.reddit.com/r/Hikvision/comments/1k...,1,3,1.00,1.746760e+09,lifelink,False,True,NaN,0,0,does software exist for


In [11]:
data.describe()

,score,num_comments,upvote_ratio,created_utc,total_awards_received,gilded
count,500.000000,500.000000,500.000000,5.000000e+02,500.0,500.0
mean,7.664000,5.408000,0.896080,1.747205e+09,0.0,0.0
std,24.386071,11.099727,0.197961,1.208906e+05,0.0,0.0
min,0.000000,0.000000,0.080000,1.746544e+09,0.0,0.0
25%,1.000000,0.000000,0.890000,1.747228e+09,0.0,0.0
50%,1.000000,2.000000,1.000000,1.747251e+09,0.0,0.0
75%,3.000000,6.000000,1.000000,1.747262e+09,0.0,0.0
max,306.000000,110.000000,1.000000,1.747266e+09,0.0,0.0


In [12]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   title                  500 non-null    object 
 1   body                   498 non-null    object 
 2   subreddit              500 non-null    object 
 3   url                    500 non-null    object 
 4   score                  500 non-null    int64  
 5   num_comments           500 non-null    int64  
 6   upvote_ratio           500 non-null    float64
 7   created_utc            500 non-null    float64
 8   author                 500 non-null    object 
 9   is_original_content    500 non-null    bool   
 10  edited                 500 non-null    bool   
 11  link_flair_text        236 non-null    object 
 12  total_awards_received  500 non-null    int64  
 13  gilded                 500 non-null    int64  
 14  search_query           500 non-null    object 
dtypes: boo

In [13]:
data.drop('link_flair_text', axis=1, inplace=True)

In [16]:
data.dropna(inplace=True)

In [18]:
# Text length and complexity
data['title_length'] = data['title'].str.len()
data['body_length'] = data['body'].fillna('').str.len()
data['title_word_count'] = data['title'].str.split().str.len()
data['body_word_count'] = data['body'].fillna('').str.split().str.len()

# Questions and urgency indicators
data['has_question_mark'] = data['title'].str.contains(r'\?').astype(int) | data['body'].fillna('').str.contains(r'\?').astype(int)

In [19]:
# Normalize engagement metrics
data['engagement_ratio'] = data['num_comments'] / (data['score'] + 1)  # Add 1 to avoid division by zero
data['award_score_ratio'] = data['total_awards_received'] / (data['score'] + 1)
data['popularity_score'] = data['score'] * data['upvote_ratio']

In [20]:
# Convert UTC to datetime and extract useful components
data['created_datetime'] = pd.to_datetime(data['created_utc'], unit='s')
data['post_hour'] = data['created_datetime'].dt.hour
data['post_day'] = data['created_datetime'].dt.day_name()
data['is_weekend'] = data['created_datetime'].dt.weekday.isin([5, 6]).astype(int)

In [21]:
# Categorize search queries
data['query_category'] = data['search_query'].map({
    'looking for software that': 'direct_request',
    'need an app that': 'direct_request',
    'app recommendation for': 'direct_request',
    'can\'t find software': 'problem_indicator',
    'wish there was an app': 'problem_indicator',
    'frustrated with current app': 'pain_point',
    # ... add the rest of your queries
})

In [24]:
data.to_csv("../data/labeled_post.csv", index=False)

In [26]:
data = pd.read_csv('../data/labeled_post.csv')

In [27]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 498 entries, 0 to 497
Data columns (total 27 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   title                  498 non-null    object 
 1   body                   498 non-null    object 
 2   subreddit              498 non-null    object 
 3   url                    498 non-null    object 
 4   score                  498 non-null    int64  
 5   num_comments           498 non-null    int64  
 6   upvote_ratio           498 non-null    float64
 7   created_utc            498 non-null    float64
 8   author                 498 non-null    object 
 9   is_original_content    498 non-null    bool   
 10  edited                 498 non-null    bool   
 11  total_awards_received  498 non-null    int64  
 12  gilded                 498 non-null    int64  
 13  search_query           498 non-null    object 
 14  title_length           498 non-null    int64  
 15  body_l